In [ ]:
from tpot import TPOTClassifier
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler

In [ ]:
# importing data
train_df = pd.read_csv("train.csv")

In [ ]:
train_df.head()

,id,A,B,C,D,E,F,G,H,I,J,K,L,M,N,O,P
0,1,b,18.42,10.415,y,p,aa,v,0.125,t,f,0,f,g,120.0,375,1
1,2,a,21.75,11.750,u,g,c,v,0.250,f,f,0,t,g,180.0,0,1
2,3,b,30.17,1.085,y,p,c,v,0.040,f,f,0,f,g,170.0,179,1
3,4,b,22.67,2.540,y,p,c,h,2.585,t,f,0,f,g,0.0,0,0
4,5,a,36.00,1.000,u,g,c,v,2.000,t,t,11,f,g,0.0,456,0


In [ ]:
train_df = train_df.drop('id',axis=1)

In [ ]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 552 entries, 0 to 551
Data columns (total 16 columns):
A    544 non-null object
B    543 non-null float64
C    552 non-null float64
D    547 non-null object
E    547 non-null object
F    545 non-null object
G    545 non-null object
H    552 non-null float64
I    552 non-null object
J    552 non-null object
K    552 non-null int64
L    552 non-null object
M    552 non-null object
N    541 non-null float64
O    552 non-null int64
P    552 non-null int64
dtypes: float64(4), int64(3), object(9)
memory usage: 69.1+ KB


In [ ]:
cat_cols = ['A', 'D', 'E', 'F', 'G', 'I', 'J', 'L', 'M']

In [ ]:
# encoding categorical column values

for header in cat_cols:
    train_df[header] = train_df[header].astype("category").cat.codes
   

In [ ]:
# creating object for normalising the continuous feature values

min_max_scaler = MinMaxScaler()
scaler_test = MinMaxScaler()

In [ ]:
ls = [x for x in cat_cols]                                  # creating a copy of list of names of categorical columns       

In [ ]:
ls.append('P')

In [ ]:
temp = train_df.drop(ls ,axis=1)                            # dataframe with continuous features only 

In [ ]:
temp.isnull().sum()

B     9
C     0
H     0
K     0
N    11
O     0
dtype: int64

In [ ]:
temp.describe()

,B,C,H,K,N,O
count,543.000000,552.000000,552.000000,552.000000,541.000000,552.000000
mean,31.364899,4.780136,2.259176,2.318841,175.162662,1087.576087
std,11.873665,4.938454,3.442564,4.297867,154.873746,5758.841945
min,13.750000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,22.500000,1.000000,0.165000,0.000000,60.000000,0.000000
50%,28.170000,2.750000,1.000000,0.000000,152.000000,3.500000
75%,37.665000,7.561250,2.750000,3.000000,268.000000,364.750000
max,80.250000,28.000000,28.500000,40.000000,1160.000000,100000.000000


In [ ]:
# imputing NaN with mean of the feature value

temp_to_normalise = temp.fillna(temp.mean())   

In [ ]:
temp.columns

Index(['B', 'C', 'H', 'K', 'N', 'O'], dtype='object')

In [ ]:
# Normalising the continuous features in the range (0,1)

scaled_inp_features = min_max_scaler.fit_transform(temp_to_normalise)
scaled_inp_features_df = pd.DataFrame(scaled_inp_features)
scaled_inp_features_df.columns = temp.columns

In [ ]:
scaled_inp_features_df.shape

(552, 6)

In [ ]:
scaled_inp_features_df.head(2)  

,B,C,H,K,N,O
0,0.070226,0.371964,0.004386,0.0,0.103448,0.00375
1,0.120301,0.419643,0.008772,0.0,0.155172,0.00000


In [ ]:
scaled_inp_features_df.index

RangeIndex(start=0, stop=552, step=1)

In [ ]:
categorical_col_df = train_df[cat_cols].copy(deep=True)                      # dataframe with categorical features only

In [ ]:
categorical_col_df.head()

,A,D,E,F,G,I,J,L,M
0,1,2,2,0,7,1,0,0,0
1,0,1,0,1,7,0,0,1,0
2,1,2,2,1,7,0,0,0,0
3,1,2,2,1,3,1,0,0,0
4,0,1,0,1,7,1,1,0,0


In [ ]:
categorical_col_df.nunique()

A     3
D     4
E     4
F    15
G    10
I     2
J     2
L     2
M     3
dtype: int64

In [ ]:
final_columns = temp.columns.values.tolist() + cat_cols

In [ ]:
type(temp.columns)

pandas.core.indexes.base.Index

In [ ]:
categorical_col_df.index

RangeIndex(start=0, stop=552, step=1)

In [ ]:
# training set
final_df = pd.concat([scaled_inp_features_df,categorical_col_df],axis=1)
final_df.columns = final_columns


In [ ]:
final_df.shape

(552, 15)

In [ ]:
train_df['P'].shape

(552,)

In [ ]:
target_series = train_df['P']
target_series.reset_index(inplace=True, drop=True)

In [ ]:
final_df.head()

,B,C,H,K,N,O,A,D,E,F,G,I,J,L,M
0,0.070226,0.371964,0.004386,0.000,0.103448,0.00375,1,2,2,0,7,1,0,0,0
1,0.120301,0.419643,0.008772,0.000,0.155172,0.00000,0,1,0,1,7,0,0,1,0
2,0.246917,0.038750,0.001404,0.000,0.146552,0.00179,1,2,2,1,7,0,0,0,0
3,0.134135,0.090714,0.090702,0.000,0.000000,0.00000,1,2,2,1,3,1,0,0,0
4,0.334586,0.035714,0.070175,0.275,0.000000,0.00456,0,1,0,1,7,1,1,0,0


In [ ]:
# concatenating target series with feature set
final_df1= pd.concat([final_df,target_series],axis=1,ignore_index=True)

# adding target column name to column names list
final_df1.columns = final_columns + ['P']          

In [ ]:
final_df1.head()

,B,C,H,K,N,O,A,D,E,F,G,I,J,L,M,P
0,0.070226,0.371964,0.004386,0.000,0.103448,0.00375,1,2,2,0,7,1,0,0,0,1
1,0.120301,0.419643,0.008772,0.000,0.155172,0.00000,0,1,0,1,7,0,0,1,0,1
2,0.246917,0.038750,0.001404,0.000,0.146552,0.00179,1,2,2,1,7,0,0,0,0,1
3,0.134135,0.090714,0.090702,0.000,0.000000,0.00000,1,2,2,1,3,1,0,0,0,0
4,0.334586,0.035714,0.070175,0.275,0.000000,0.00456,0,1,0,1,7,1,1,0,0,0


In [ ]:
final_df1.shape

(552, 16)

In [ ]:
final_df1.columns.values

array(['B', 'C', 'H', 'K', 'N', 'O', 'A', 'D', 'E', 'F', 'G', 'I', 'J',
       'L', 'M', 'P'], dtype=object)

In [ ]:
final_df1.head()

,B,C,H,K,N,O,A,D,E,F,G,I,J,L,M,P
0,0.070226,0.371964,0.004386,0.000,0.103448,0.00375,1,2,2,0,7,1,0,0,0,1
1,0.120301,0.419643,0.008772,0.000,0.155172,0.00000,0,1,0,1,7,0,0,1,0,1
2,0.246917,0.038750,0.001404,0.000,0.146552,0.00179,1,2,2,1,7,0,0,0,0,1
3,0.134135,0.090714,0.090702,0.000,0.000000,0.00000,1,2,2,1,3,1,0,0,0,0
4,0.334586,0.035714,0.070175,0.275,0.000000,0.00456,0,1,0,1,7,1,1,0,0,0


In [ ]:
# indices of categorical columns

cat_dims = [final_df1.columns.get_loc(i) for i in cat_cols]   


In [ ]:
cat_dims

[6, 7, 8, 9, 10, 11, 12, 13, 14]

In [ ]:
encoded_set = pd.get_dummies(final_df1, columns=cat_cols )

In [ ]:
encoded_set.shape

(552, 52)

In [ ]:
split_limit_index = int(0.8*train_df.shape[0])

In [ ]:
sample_train = encoded_set[:split_limit_index].drop('P',axis=1)
sample_val = encoded_set[split_limit_index:].drop('P',axis=1)

In [ ]:
sample_train.head()

,B,C,H,K,N,O,A_-1,A_0,A_1,D_-1,...,G_8,I_0,I_1,J_0,J_1,L_0,L_1,M_0,M_1,M_2
0,0.070226,0.371964,0.004386,0.000,0.103448,0.00375,0,0,1,0,...,0,0,1,1,0,1,0,1,0,0
1,0.120301,0.419643,0.008772,0.000,0.155172,0.00000,0,1,0,0,...,0,1,0,1,0,0,1,1,0,0
2,0.246917,0.038750,0.001404,0.000,0.146552,0.00179,0,0,1,0,...,0,1,0,1,0,1,0,1,0,0
3,0.134135,0.090714,0.090702,0.000,0.000000,0.00000,0,0,1,0,...,0,0,1,1,0,1,0,1,0,0
4,0.334586,0.035714,0.070175,0.275,0.000000,0.00456,0,1,0,0,...,0,0,1,0,1,1,0,1,0,0


In [ ]:
sample_train.shape

(441, 51)

In [ ]:
train_label = final_df1['P'][:split_limit_index]
val_label = final_df1['P'][split_limit_index:]

In [ ]:
sample_train.head()

,B,C,H,K,N,O,A_-1,A_0,A_1,D_-1,...,G_8,I_0,I_1,J_0,J_1,L_0,L_1,M_0,M_1,M_2
0,0.070226,0.371964,0.004386,0.000,0.103448,0.00375,0,0,1,0,...,0,0,1,1,0,1,0,1,0,0
1,0.120301,0.419643,0.008772,0.000,0.155172,0.00000,0,1,0,0,...,0,1,0,1,0,0,1,1,0,0
2,0.246917,0.038750,0.001404,0.000,0.146552,0.00179,0,0,1,0,...,0,1,0,1,0,1,0,1,0,0
3,0.134135,0.090714,0.090702,0.000,0.000000,0.00000,0,0,1,0,...,0,0,1,1,0,1,0,1,0,0
4,0.334586,0.035714,0.070175,0.275,0.000000,0.00456,0,1,0,0,...,0,0,1,0,1,1,0,1,0,0


In [ ]:
sample_val.head()

,B,C,H,K,N,O,A_-1,A_0,A_1,D_-1,...,G_8,I_0,I_1,J_0,J_1,L_0,L_1,M_0,M_1,M_2
441,0.253083,0.380893,0.002982,0.300,0.111207,0.00003,0,1,0,0,...,0,1,0,0,1,0,1,1,0,0
442,0.101504,0.086250,0.070175,0.275,0.172414,0.03000,0,0,1,0,...,0,0,1,0,1,0,1,1,0,0
443,0.130376,0.401786,0.026316,0.100,0.000000,0.00321,0,1,0,0,...,0,0,1,0,1,1,0,1,0,0
444,0.207970,0.116071,0.178421,0.050,0.318103,0.00001,0,0,1,0,...,0,1,0,0,1,0,1,1,0,0
445,0.047669,0.017857,0.005789,0.150,0.206897,0.00035,0,1,0,0,...,0,1,0,0,1,0,1,1,0,0


In [ ]:
train_label.shape

(441,)

In [ ]:
val_label.shape

(111,)

In [ ]:
train_set = sample_train.drop('P',axis=1)
val_set = sample_val.drop('P',axis=1)

In [ ]:
train_label = sample_train['P']
val_label = sample_val['P']

Applying TPOT classifier

In [ ]:
# ,scoring='neg_log_loss' ,random_state = 2,periodic_checkpoint_folder:    (to be added below)

In [ ]:
tpot1 = TPOTClassifier(generations=100, population_size=100, verbosity=2)

In [ ]:
%time tpot1.fit(sample_train.values, train_label)

Optimization Progress:   2%|▏         | 200/10100 [01:44<1:44:25,  1.58pipeline/s]

Generation 1 - Current best internal CV score: 0.8844117267209093


Optimization Progress:   3%|▎         | 300/10100 [03:05<1:59:06,  1.37pipeline/s]

Generation 2 - Current best internal CV score: 0.8889061087433812


Optimization Progress:   4%|▍         | 400/10100 [04:36<1:22:38,  1.96pipeline/s]

...

Generation 98 - Current best internal CV score: 0.91372852899393


Optimization Progress:  99%|█████████▉| 10000/10100 [2:54:07<01:01,  1.62pipeline/s]

Generation 99 - Current best internal CV score: 0.91372852899393


Generation 100 - Current best internal CV score: 0.91372852899393

Best pipeline: RandomForestClassifier(GradientBoostingClassifier(GradientBoostingClassifier(Normalizer(MaxAbsScaler(input_matrix), norm=max), learning_rate=0.001, max_depth=2, max_features=0.05, min_samples_leaf=12, min_samples_split=20, n_estimators=100, subsample=0.35000000000000003), learning_rate=0.01, max_depth=5, max_features=0.55, min_samples_leaf=15, min_samples_split=20, n_estimators=100, subsample=0.7500000000000001), bootstrap=False, criterion=entropy, max_features=0.1, min_samples_leaf=13, min_samples_split=15, n_estimators=100)
CPU times: user 2h 40min 13s, sys: 1min 51s, total: 2h 42min 4s
Wall time: 2h 55min 35s


TPOTClassifier(config_dict={'sklearn.kernel_approximation.RBFSampler': {'gamma': array([0.  , 0.05, 0.1 , 0.15, 0.2 , 0.25, 0.3 , 0.35, 0.4 , 0.45, 0.5 ,
       0.55, 0.6 , 0.65, 0.7 , 0.75, 0.8 , 0.85, 0.9 , 0.95, 1.  ])}, 'sklearn.cluster.FeatureAgglomeration': {'linkage': ['ward', 'complete', 'average'], 'affin..., 0.6 , 0.65, 0.7 , 0.75, 0.8 , 0.85, 0.9 , 0.95, 1.  ])}, 'sklearn.preprocessing.RobustScaler': {}},
        crossover_rate=0.1, cv=5, disable_update_check=False,
        early_stop=None, generations=100, max_eval_time_mins=5,
        max_time_mins=None, memory=None, mutation_rate=0.9, n_jobs=1,
        offspring_size=100, periodic_checkpoint_folder=None,
        population_size=100, random_state=None, scoring=None,
        subsample=1.0, verbosity=2, warm_start=False)

In [ ]:
print(tpot1.score(sample_val.values, val_label.values))
tpot1.export('tpot_Binary_classification_pipeline.py')

0.8378378378378378


True

#### References
* https://epistasislab.github.io/tpot/